In [22]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

In [23]:
df=pd.read_csv("BankNote_Authentication.csv")

In [24]:
df=df.sample(frac=1, random_state=42).reset_index(drop=True)
#veri setinden karışık halde sample almak ve bunların indexlerini çıkarmak için 

In [25]:
x,y= df.iloc[:,:-1], df.iloc[:,-1]

In [26]:
x=x.to_numpy()

In [27]:
y=y.to_numpy().reshape(-1,1)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=123, stratify=y)

In [29]:
def initialize_parameters(layer_no, n_x, n_h ,n_y=1):
    

    #n_h: hücre sayısı
    #n_x: nitelik sayısı
    #n_y: output sayısı
    #burada her bir layer için parametreleri initialize ediyoruz
    
    np.random.seed(42)
    if(layer_no==2):
        w1=np.random.randn(n_h,n_x)*0.01
        b1=np.zeros((n_h,1))       
        w2=np.random.randn(n_y,n_h)*0.01
        b2=np.zeros((n_y,1))
        parameters={
             "w1":w1,
             "b1":b1,
             "w2":w2,
             "b2":b2
        }
    if(layer_no==3):
        w1=np.random.randn(n_h,n_x)*0.01
        b1=np.zeros((n_h,1))
        w2=np.random.randn(n_h,n_h)*0.01
        b2=np.zeros((n_h,1))
        w3=np.random.randn(n_y,n_h)*0.01
        b3=np.zeros((n_y,1))
        parameters={
             "w1":w1,
             "b1":b1,
             "w2":w2,
             "b2":b2,
             "w3":w3,
             "b3":b3
            }
 
    return parameters

In [38]:
def relu(x):
  return np.maximum(0, x)

In [40]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-1 * Z))

In [56]:
def forward_propagation(layer_no, X, parameters):
    #burada layerların teker teker ileri yayılımını yapıyoruz, layer3 olan: 1. ve 2. hidden layer için relu,
    # output layer için sigmoid fonskiyonu kullanıldı
    # layer2 olan: 1. hidden layer için relu,
    # output layer için sigmoid fonskiyonu kullanıldı
    if(layer_no==2):
        w1 = parameters["w1"]
        b1 = parameters["b1"]
        w2 = parameters["w2"]
        b2 = parameters["b2"]
        
        Z0 = np.dot(w1,X.T) 
        Z1= Z0 + b1 
        A1 = relu(Z1)
        Z22 = np.dot(A1.T,w2.T) 
        Z2= Z22.T + b2 
        A2 = sigmoid(Z2)  
        output=A2
        
        cache = {
            "Z0" : Z0,
            "Z1" : Z1,
            "A1" : A1,
            "Z2" : Z2,
            "A2" : A2
        }
    if(layer_no==3):
        w1 = parameters["w1"]
        b1 = parameters["b1"]
        w2 = parameters["w2"]
        b2 = parameters["b2"]
        w3 = parameters["w3"]
        b3 = parameters["b3"]
        
        Z0 = np.dot(w1,X.T) 
        Z1= Z0 + b1 
        A1 = relu(Z1)
        Z22 = np.dot(A1.T,w2) 
        Z2= Z22.T + b2 
        A2 = relu(Z2)  
        Z33 = np.dot(w3, A2) 
        Z3= Z33 + b3 
        A3 = sigmoid(Z3) 
        output=A3
        cache = {
            "Z0" : Z0,
            "Z1" : Z1,
            "A1" : A1,
            "Z2" : Z2,
            "A2" : A2,
            "Z3" : Z3,
            "A3" : A3
        }
    
    return output, cache

In [42]:
def compute_cost(y_prediction, Y):
    m = y_prediction.shape[1]
    c0 =np.dot(sigmoid(np.log(y_prediction)), Y)
    c1=np.dot(sigmoid(np.log(1 - y_prediction)), (1 - Y))
    cost=(c0+c1)/m
    cost = float(np.squeeze(cost))
    return cost

In [43]:
def backpropagation(layer_no, X, Y, cache, parameters):
    
    m = X.shape[0]
    if(layer_no==2):
        w1 = parameters["w1"]
        w2 = parameters["w2"]
      
        A1 = cache["A1"]
        A2 = cache["A2"]
       
        dZ2 = A2.T - Y 
        dW2 = np.dot(dZ2.T, A1.T) / m 
        db2 = np.sum(dZ2, axis = 0, keepdims = True) / m 
        dZ1 = np.dot(dZ2, w2) * relu(1 - np.power(A1, 2)).T 
        dW1 = np.dot(dZ1.T, X) / m 
        db1 = np.sum(dZ1, axis = 0, keepdims = True) / m 
    
        grads = {
            "dW1" : dW1,
            "dW2" : dW2,
            "db1" : db1,
            "db2" : db2
          
        }
        
    if(layer_no==3):
        w1 = parameters["w1"]
        w2 = parameters["w2"]
        w3 = parameters["w3"]
        A1 = cache["A1"]
        A2 = cache["A2"]
        A3 = cache["A3"]

        dZ3 = A3.T - Y 
        dW3 = np.dot(dZ3.T, A2.T) / m 
        db3 = np.sum(dZ3, axis = 0, keepdims = True) / m 
        dZ2 = np.dot(dZ3, w3) * relu(1 - np.power(A2, 2)).T 
        dW2 = np.dot(dZ2.T, A1.T) / m 
        db2 = np.sum(dZ2, axis = 0, keepdims = True) / m 
        dZ1 = np.dot(dZ2, w2) * relu(1 - np.power(A1, 2)).T 
        dW1 = np.dot(dZ1.T, X) / m 
        db1 = np.sum(dZ1, axis = 0, keepdims = True) / m 
    
        grads = {
            "dW1" : dW1,
            "dW2" : dW2,
            "dW3" : dW3,
            "db1" : db1,
            "db2" : db2,
            "db3" : db3
        }
    
    return grads

In [44]:
def update_parameters(layer_no, parameters, grads, learning_rate = 0.01):
    if(layer_no==2):
        w1 = parameters["w1"]
        b1 = parameters["b1"]
        w2 = parameters["w2"]
        b2 = parameters["b2"]
     

        dW1 = grads["dW1"]
        dW2 = grads[ "dW2" ]
        db1= grads["db1"]
        db2 = grads["db2"] 
       
    
        w1 -= learning_rate * dW1
        b1 -= learning_rate * db1.T
        w2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        
    
        parameters = {
            "w1" : w1,
            "b1" : b1,
            "w2" : w2,
            "b2" : b2
        }
        
    if(layer_no==3):
        w1 = parameters["w1"]
        b1 = parameters["b1"]
        w2 = parameters["w2"]
        b2 = parameters["b2"]
        w3 = parameters["w3"]
        b3 = parameters["b3"]

        dW1 = grads["dW1"]
        dW2 = grads[ "dW2" ]
        dW3 = grads["dW3" ]
        db1= grads["db1"]
        db2 = grads["db2"] 
        db3 = grads[ "db3" ]
    
        w1 -= learning_rate * dW1
        b1 -= learning_rate * db1.T
        w2 -= learning_rate * dW2
        b2 -= learning_rate * db2.T
        w3 -= learning_rate * dW3
        b3 -= learning_rate * db3
    
        parameters = {
            "w1" : w1,
            "b1" : b1,
            "w2" : w2,
            "b2" : b2,
            "w3" : w3,
            "b3" : b3
        }
    return parameters

In [45]:
def nn_model(layer_no, X, Y, n_x, n_h, n_y, n_steps = 1000, print_cost = True):
    parameters = initialize_parameters(layer_no, n_x, n_h, n_y)
    
    for i in range(0, n_steps):
        y_prediction, cache = forward_propagation(layer_no, X, parameters)
        cost = compute_cost(y_prediction, Y)
        grads = backpropagation(layer_no, X, Y, cache, parameters)
        parameters = update_parameters(layer_no, parameters, grads)
        
        if print_cost and i % 1000 == 0:
            print("cost: %i , %f" %(i,cost))
    
    return parameters

In [58]:
parameters_2 = nn_model(2,x_train, y_train, x_train.shape[1], n_h = 5, n_y = 1, n_steps = 1000)
parameters_3 = nn_model(3,x_train, y_train, x_train.shape[1], n_h = 5, n_y = 1, n_steps = 1000)


cost: 0 , 0.333319
cost: 0 , 0.333333


In [60]:
def predict(layer_no, parameters, X):
    output_pred, cache = forward_propagation(layer_no, X, parameters)
    predicts = output_pred > 0.44942162
    return predicts

In [64]:
predicts_2 = predict(2,parameters_2, x_test)
print(x_test.shape,predicts_2.shape)
predicts_3 = predict(3,parameters_3, x_test)
print(predicts_3.shape)

(275, 4) (1, 275)
(1, 275)


In [66]:
y_true = y_test.flatten().T#(275,1)'lik olan boyutu sadece 275 yapmak için kullanıldı
y_pred_2 = predicts_2.flatten().T
acc = accuracy_score(y_true, y_pred_2)
precision = precision_score(y_true, y_pred_2, average='binary') 
recall = recall_score(y_true, y_pred_2, average='binary')
f1 = f1_score(y_true, y_pred_2, average='binary')
conf_matrix = confusion_matrix(y_true, y_pred_2)
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_2)) 

Accuracy: 0.8691
Precision: 0.8209
Recall: 0.9016
F1 Score: 0.8594

Confusion Matrix:
[[129  24]
 [ 12 110]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.88       153
           1       0.82      0.90      0.86       122

    accuracy                           0.87       275
   macro avg       0.87      0.87      0.87       275
weighted avg       0.87      0.87      0.87       275



In [68]:
y_true = y_test.flatten().T#(275,1)'lik olan boyutu sadece 275 yapmak için kullanıldı
y_pred_3 = predicts_3.flatten().T
acc = accuracy_score(y_true, y_pred_3)
precision = precision_score(y_true, y_pred_3, average='binary') 
recall = recall_score(y_true, y_pred_3, average='binary')
f1 = f1_score(y_true, y_pred_3, average='binary')
conf_matrix = confusion_matrix(y_true, y_pred_3)
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_3)) 

Accuracy: 0.7491
Precision: 0.6879
Recall: 0.7951
F1 Score: 0.7376

Confusion Matrix:
[[109  44]
 [ 25  97]]

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.71      0.76       153
           1       0.69      0.80      0.74       122

    accuracy                           0.75       275
   macro avg       0.75      0.75      0.75       275
weighted avg       0.76      0.75      0.75       275

